# 📌 PyTorch Eğitim Döngüsü Pipeline

---

## Eğitim Döngüsü Akışı

```text
+-------------------+
|  Veri Hazırlama    |
|  (DataLoader)      |
+---------+---------+
          |
          v
+-------------------+
|  Model.train()     | ← Eğitim moduna geç
+---------+---------+
          |
          v
+-------------------+
|  Mini-Batch Loop   | ← Her batch için
+---------+---------+
          |
          v
+-------------------------------+
|  1️⃣ Forward Pass              |
|     inputs → model → outputs   |
+-------------------------------+
          |
          v
+-------------------------------+
|  2️⃣ Loss Hesaplama           |
|     outputs + target → loss   |
+-------------------------------+
          |
          v
+-------------------------------+
|  3️⃣ Backward Pass            |
|     loss.backward()           |
+-------------------------------+
          |
          v
+-------------------------------+
|  4️⃣ Parametre Güncelleme      |
|     optimizer.step()           |
+-------------------------------+
          |
          v
+-------------------------------+
|  Epoch Sonu                   |
|  - Ortalama Loss hesapla      |
|  - Scheduler.step(loss)       |
+-------------------------------+
          |
          v
+-------------------------------+
|  model.eval()                 |
|  - Gradient kapalı             |
|  - Validation/Test seti        |
|  - Çıkışlar -> sınıflar (0/1)|
+-------------------------------+


# 📌 Eğitim Döngüsü Açıklamaları



### 🔹 DataLoader
- Veriyi **mini-batch**’lere böler.
- Bellek kullanımını optimize eder ve eğitim hızını artırır.



### 🔹 model.train()
- Modeli **eğitim moduna** geçirir.
- Dropout ve BatchNorm gibi katmanlar eğitim davranışına geçer.



### 🔹 Forward Pass
- Model **tahminlerini üretir**.
- Input verileri ağırlıklardan geçirerek output hesaplanır.



### 🔹 Loss Hesaplama
- Tahminler ile gerçek değerler arasındaki **kayıp (loss)** hesaplanır.
- Örnek: BCEWithLogitsLoss, CrossEntropyLoss



### 🔹 Backward Pass
- `loss.backward()` ile **gradyanlar** hesaplanır.
- Tüm ağırlıklar için türevler bulunur.



### 🔹 Parametre Güncelleme
- Optimizer ile ağırlıklar **güncellenir**.
- Örnek: `optimizer.step()`



### 🔹 Scheduler
- Öğrenme oranını **kayba göre otomatik ayarlar**.
- Örnek: ReduceLROnPlateau, StepLR



### 🔹 model.eval()
- Modeli **değerlendirme moduna** geçirir.
- Dropout kapalı, BatchNorm sabit.
- Validation veya test seti ile performans ölçülür.



### 🔹 Sınıflandırma
- Çıkış olasılıkları (0-1) **eşik değere göre sınıflandırılır**.
- Binary classification için genellikle **0.5 eşik** kullanılır.


#### ŞİMDİ GELİN BUNLARI KOD OLARAK NASIL AÇIKLAYACAĞIMIZA BAKALIM

---

- Modeli, loss ve optimizer'ı tanımladığımızı varsayıyoruz:
- model, criterion, optimizer, scheduler hazır

 # ------------------------
    # 1️⃣ Eğitim Adımı
    # ------------------------


- Modeli, loss ve optimizer'ı tanımladığımızı varsayıyoruz:
- model, criterion, optimizer, scheduler hazır

In [ ]:
num_epochs = 50
train_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    # ------------------------
    # 1️⃣ Eğitim Adımı
    # ------------------------
    model.train()  # Dropout ve BatchNorm eğitim moduna geçer
    running_loss = 0.0
    
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()            # Önceki gradyanları sıfırla
        outputs = model(X_batch)         # Forward pass
        loss = criterion(outputs, y_batch)  # Loss hesaplama
        loss.backward()                  # Backward pass
        optimizer.step()                 # Parametre güncelleme
        running_loss += loss.item() * X_batch.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)
    
    # Scheduler güncellemesi (ReduceLROnPlateau)
    scheduler.step(epoch_loss)
    
    # ------------------------
    # 2️⃣ Değerlendirme Adımı
    # ------------------------
    model.eval()  # Dropout kapalı, BatchNorm sabit
    with torch.no_grad():  # Gradient kapalı
        y_pred = model(X_test)
        y_pred_label = (torch.sigmoid(y_pred) >= 0.5).float()  # 0.5 eşik ile sınıflandırma
        accuracy = (y_pred_label == y_test).sum().item() / y_test.size(0)
        test_accuracies.append(accuracy)
    
    # ------------------------
    # Epoch Sonu Bilgisi
    # ------------------------
    print(f"Epoch [{epoch+1}/{num_epochs}] | Loss: {epoch_loss:.4f} | Test Accuracy: {accuracy*100:.2f}%")


In [ ]:
import matplotlib.pyplot as plt

# 📊 Eğitim & Değerlendirme Grafikleri
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

# 1️⃣ Loss Grafiği
axs[0].plot(train_losses, label="Train Loss")
axs[0].set_title("Eğitim Kaybı (Loss)")
axs[0].set_xlabel("Epoch")
axs[0].set_ylabel("Loss")
axs[0].legend()
axs[0].grid(True)

# 2️⃣ Accuracy Grafiği
axs[1].plot(test_accuracies, label="Test Accuracy", color="green")
axs[1].set_title("Test Doğruluğu (Accuracy)")
axs[1].set_xlabel("Epoch")
axs[1].set_ylabel("Accuracy")
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()
